# Neural Network Structure
## Objective:
Train a Neural Network algorithm using the .csv file dataset. The data is already cleaned. 

### Setup

In [1]:
# Import the required libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers.legacy import RMSprop
from sklearn.model_selection import KFold

#### Dataset
I will load the dataset, preview it, and output the number of tuples and features.

In [2]:
# Load data from CSV file
df = pd.read_csv('../data.csv')

In [3]:
# Display the top 10 rows of the dataset to ensure it was loaded
df.head(10)

,X1,X2,X3,X4,X5,Y
0,23.180,27.2720,426.0,721.250000,0.004793,1
1,23.150,27.2675,429.5,714.000000,0.004783,1
2,23.150,27.2450,426.0,713.500000,0.004779,1
3,23.150,27.2000,426.0,708.250000,0.004772,1
4,23.100,27.2000,426.0,704.500000,0.004757,1
5,23.100,27.2000,419.0,701.000000,0.004757,1
6,23.100,27.2000,419.0,701.666667,0.004757,1
7,23.100,27.2000,419.0,699.000000,0.004757,1
8,23.100,27.2000,419.0,689.333333,0.004757,1
9,23.075,27.1750,419.0,688.000000,0.004745,1


In [4]:
# Get the number of tuples and features in the dataset
df.shape

(8143, 6)

The dataset has 5 features and 1 output, which can be one of two possible classes: 0 or 1.

### Data Preprocessing
Since this is a larger dataset, I will perform some feature engineering. First, I will compute the correlation between each feature and the target. Then, I will drop the two features with the lowest correlation.

In [5]:
# Find and output correlation
correlations = df.corr()['Y'].drop('Y')
print(correlations)

X1    0.538220
X2    0.132964
X3    0.907352
X4    0.712235
X5    0.300282
Name: Y, dtype: float64


In [6]:
# Drop the two features with lowest correlation
df = df.drop(columns=['X2', 'X5'])

### Split the Data into Training and Testing Sets
First, I will divide the data into features and target. Then, I will further divide these into training and testing sets. I will use 80% of the data for training and 20% for testing.

In [7]:
# Divide data into features and target
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

### Create the Neural Network Structure
The neural network I will create will have an input layer, two hidden layers, and an output layer. The input and hidden layers will use ReLU Activation. The output layer will use Sigmoid Activation since this is a binary classification task. Dropout regulization layers will be used to prevent overfitting. The model will be created through a function, so that it other models can be made iteratively and used in kfold validation.

In [8]:
# Define the RMS optimizer
rms = RMSprop(learning_rate=0.002, rho=0.85, epsilon=1e-08, decay=0.0)

def create_model():
    model = Sequential()

    # Input Layer
    model.add(Dense(10, input_dim=3, activation='relu'))
    model.add(Dropout(0.2))  # Drop 20% of neurons

    # Hidden Layers
    model.add(Dense(40, activation='relu'))
    model.add(Dropout(0.2))  # Drop 20% of neurons

    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.2))  # Drop 20% of neurons

    # Output Layer
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=rms, metrics=['accuracy'])
    return model

# Creating a sample model to show the structure
sample_model = create_model()
sample_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                40        
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 40)                440       
                                                                 
 dropout_1 (Dropout)         (None, 40)                0         
                                                                 
 dense_2 (Dense)             (None, 50)                2050      
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 5

### Training and Testing with KFold Cross-Validator
To provide a better evaluation of the model, KFold will be used to divde the data into a specifed number of folds: 10 in this case. During training, one of the folds will be kept for evaluating the model while the rest are used for training. By using a for loop, the program will create train 10 different models, each of them training and testing on different parts of the dataset.

In [9]:
# Defining a 10-fold cross validation 
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# List that will hold the scores 
cvscores = []

for train, test in kfold.split(X, y):
    # Create model
    model = create_model()

    # Fit the model
    model.fit(X[train], y[train], epochs=50, batch_size=100, verbose=1)

    # Evaluate the model using the testing data
    scores = model.evaluate(X[test], y[test], verbose=1)

    # Print out the accuracy of each iteration
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

    # Append the score to the list
    cvscores.append(scores[1] * 100)

# Print out the average accuracy and the standard deviation
print("Average Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/50
74/74 [==============================] - 1s 1ms/step - loss: 5.5354 - accuracy: 0.7208
Epoch 2/50
74/74 [==============================] - 0s 1ms/step - loss: 1.0476 - accuracy: 0.8203
Epoch 3/50
74/74 [==============================] - 0s 1ms/step - loss: 0.3871 - accuracy: 0.8724
Epoch 4/50
74/74 [==============================] - 0s 1ms/step - loss: 0.2723 - accuracy: 0.8956
Epoch 5/50
74/74 [==============================] - 0s 1ms/step - loss: 0.2273 - accuracy: 0.9049
Epoch 6/50
74/74 [==============================] - 0s 1ms/step - loss: 0.2163 - accuracy: 0.9150
Epoch 7/50
74/74 [==============================] - 0s 1ms/step - loss: 0.1968 - accuracy: 0.9206
Epoch 8/50
74/74 [==============================] - 0s 1ms/step - loss: 0.1830 - accuracy: 0.9327
Epoch 9/50
74/74 [==============================] - 0s 1ms/step - loss: 0.1640 - accuracy: 0.9438
Epoch 10/50
74/74 [==============================] - 0s 1ms/step - loss: 0.1570 - accuracy: 0.9548
Epoch 11/50
74/74 [

### Summary
After training and evaluating the model, it has performed with an average accuracy of 98.54% with the highest accuracy reached at 99.39%. Overfitting and underfitting did not present an issue in this case due to the preventative measures taken by using Dropoff and Kfold cross validation.